In [1]:
from collections import defaultdict


In [ ]:


mm = open("Mmgene.out")
alts_to_genes = {}
genes_to_alts = {}
nm_mapping = {}
nm_mapping_rev = {}
for line in mm:
    sp = line.strip().split("\t")
    nm_mapping[sp[4].split(".")[0]] = sp[7]
    nm_mapping_rev[sp[7]] = sp[4].split(".")[0]
    for alt in sp[14].split("|"):
        if alt in alts_to_genes:
            lst = alts_to_genes[alt]
            lst.append(sp[7])
            alts_to_genes[alt] = lst
        else:
            alts_to_genes[alt] = [sp[7]]
        gene = sp[7]
        if gene in genes_to_alts:
            lst = genes_to_alts[gene]
            lst.append(alt)
            genes_to_alts[gene] = lst
        else:
            genes_to_alts[gene] = [alt]
    

In [2]:
biomart = open("mart_full.txt")
biomart_gene_to_nm = {}
biomart_nm_to_gene = {}
for line in biomart:

    sp = line.strip().split("\t")
    if len(sp) >= 23:
        gene = sp[6]
        nm = sp[22]

    
        if gene in biomart_gene_to_nm:
            lm = biomart_gene_to_nm[gene]
            lm[nm] = True
            biomart_gene_to_nm[gene] = lm
        else:
            biomart_gene_to_nm[gene] = {nm: True}
    
        if nm in biomart_nm_to_gene:
            lm = biomart_nm_to_gene[nm]
            lm[gene] = True
            biomart_nm_to_gene[nm] = lm
        else:
            biomart_nm_to_gene[nm] = {gene: True}

In [56]:
# 11287   Pzp     NM_007376.4     NC_000072.6     128522753       antisense       TTACCTCAATATAAACGACA    GTACTTACCTCAATATAAACGACAGGGAAC  GGG     5       0.707
brie = open("broadgpp-brie-library-contents.txt")
brie_gene_to_nm = {}
brie_gene_to_guides = defaultdict(list)

brie_nm_to_gene = {}
for line in brie:
    sp = line.strip().split("\t")
    gene = sp[1]
    nm = sp[2].split(".")[0]
    guide_list = brie_gene_to_guides[gene]
    guide_list.append([sp[6],sp[7]])
    brie_gene_to_guides[gene] = guide_list
    
    if gene in brie_gene_to_nm:
        lm = brie_gene_to_nm[gene]
        lm.append(nm)
        brie_gene_to_nm[gene] = lm
        
    else:
        brie_gene_to_nm[gene] = [nm]

    if nm in brie_nm_to_gene:
        lm = brie_nm_to_gene[nm]
        lm.append(gene)
        brie_nm_to_gene[nm] = lm
    else:
        brie_nm_to_gene[nm] = [gene]
        
    
    

In [ ]:
lookup_file = open("2024_12_23_mouse_genes_to_lookup.txt")
header = lookup_file.readline()
lookups = []
for line in lookup_file:
    tk = line.strip()
    if tk != "" and tk != "NA":
        lookups.append(line.strip())
lookups

In [ ]:
remapping = {x: "------" for x in lookups}

for lookup in lookups:
    found = False
    if lookup in nm_mapping_rev and nm_mapping_rev[lookup] in brie_nm_to_gene:
        print(lookup, " by NM to ", brie_nm_to_gene[nm_mapping_rev[lookup]])
        found = True
        remapping[lookup] = brie_nm_to_gene[nm_mapping_rev[lookup]]
    elif not found and lookup in alts_to_genes:
        for alt in alts_to_genes[lookup]:
            if alt in nm_mapping_rev and nm_mapping_rev[alt] in brie_nm_to_gene:
                print(alt, " by NM to ", brie_nm_to_gene[nm_mapping_rev[alt]]) 
                found = True
                remapping[lookup] = brie_nm_to_gene[nm_mapping_rev[alt]]

        print(lookup, " in alts")
    elif not found and lookup in genes_to_alts:
        print(lookup, " in genes")
        for alt in genes_to_alts[lookup]:
            if alt in nm_mapping_rev and nm_mapping_rev[alt] in brie_nm_to_gene:
                print(alt, " by NM to ", brie_nm_to_gene[nm_mapping_rev[alt]]) 
                found = True
                remapping[lookup] = brie_nm_to_gene[nm_mapping_rev[alt]]
    elif not found and lookup in biomart_gene_to_nm:
        print("trying biomart for ",lookup, biomart_gene_to_nm[lookup])
        for alt in biomart_gene_to_nm[lookup].keys():
            if alt in brie_nm_to_gene:
                print("Found brie to NM: ",brie_nm_to_gene[alt])
                for key in brie_nm_to_gene[alt]:
                    remapping[lookup] = key
    else:
        print("No answer for ",lookup)

In [ ]:
remapping

In [ ]:
# make the ultimate guide ordering sheet(s)
# first a spreadsheet of the genes to corrected name with the resulting guide sequence

CTAGCGCGTTAACAACAACC
GAGGTTCAGCAACAAGGTGG
TCCTTAATCCAACCGATTCC
AACTCTCGCCATAAGAGCAC
AAGACTGAGACAATACGTCC
CTCTATACACAATGGTAACG
ACAGTTATGCGTCCACAGCT
CGAGAAGGAAACCGCAAGAC
ACGCCTCTTACGGACACCTA
CCGGTAGTTCAGCTACCATG
AGGAATTGCCTGGTGCATAA
AGTGTGGTCCGCGTCGTGAA
ATCTTGGAGTGTAGAATCCT
CCACTCGAACGTTACTCGGT















In [47]:


mk_list = open("MRK_List1.rpt")
mrk_map = defaultdict(list)

for line in mk_list:
    tks = line.strip().split("\t")
    if len(tks) > 11:
        known = tks[6]
        alts = tks[11].split("|")
        alts.append(known)
        alts = list(filter(lambda x: x.strip() != "", alts))
        for alt in alts:
            lst = mrk_map[alt]
            lst.extend(alts)
            mrk_map[alt] = lst


In [53]:
teichman_mouse = {}
tfile = open("teichman_mouse_v2_grnas.txt")
header = tfile.readline()
for line in tfile:
    sp = line.strip().split()
    teichman_mouse[sp[1]] = [sp[0],sp[2]]

In [52]:
remap = open("remapped_mouse_2024_12_23.txt","w")
remap.write("set\tgene\tremapped_name\tremapped_guide\tother_candidates\n")
for lookup in lookups:
    mapped_entry = None
    if lookup in mrk_map:
        new_named = None
        for new_name in mrk_map[lookup]:
            if new_name in teichman_mouse:
                new_named = new_name
        if new_named != None:
            mapped_entry = "teichman\t" + lookup + "\t" + new_named + "\t" +  str(mrk_map[lookup]) + "\n"
    if lookup in brie and mapped_entry == None:
        
            
    else:
        remap.write("FALSE\t" + lookup + "\tNONE\tNONE\n")

remap.close()